# Assignment 1; Part 1: Scrapping the Wikipedia and Population the database # 

For the scrapping framework I have chosen Scrapy, since it have been seen as a more powerful tool than BeatifoulSoup. \


In [1]:
import scrapy
from scrapy.crawler import CrawlerProcess

## Scrapy Spider ## 
To scrap the Wikipedia page, in Scrapy we have to define so-called "Spider", which will go through the needed pages. \
- First of all, I provide a name for this spider, to be invoked in future.
- Secondly, I provide the start url page. I plan to scrap top-50 table, and extract from this table links to the movie main page itself, from which I will extract additional information.
- Then I create two function: 'parse' and 'parse_film_details'. The 'parse' function is default function which will be called on every new page request; while 'parse_film_details' is invoked on the movie main page request;


In [2]:

class FilmDetailSpider(scrapy.Spider):
    name = "film_details" # the name of the spider, to be invoked in future.

    start_urls = ["https://en.wikipedia.org/wiki/List_of_highest-grossing_films"] # the url from which we will start scrapping
    
    custom_settings = {
        'FEEDS': { 'data.json': { 'format': 'json', 'encoding': 'utf-8', 'overwrite': True}}
    } # these settings is needed, for example, to save the output of scrapping in json file.

    def parse(self, response):
        table = response.xpath("//table[contains(@class, 'wikitable')][1]/tbody") # go to the table path on the web-page
        rows = table.xpath("./tr")[1:]  # Skip header row

        for row in rows:
            # Extract title and link
            title_element = row.xpath("./th[@scope='row']//i/a | ./th[@scope='row']//span/i/a")  # Handle both cases
            title = title_element.xpath("text()").get()
            link = title_element.xpath("@href").get()

            # Extract year (handle both direct text and data-sort-value cases)
            year_column = row.xpath("./td[4]")
            year = year_column.xpath("text()").get() or year_column.xpath("@data-sort-value").get()

            if title and link:
                # if title and link present, follow the link to the main page about the movie, invoke different parses
                yield response.follow(link, self.parse_film_details, meta={'title': title, 'year': year})




    def parse_film_details(self, response):
        # these values came from table, which we scrapped earlier, so just move here
        title = response.meta['title']
        year = response.meta['year']

        directors = response.xpath("//th[contains(text(),'Directed by')]/following-sibling::td//li//text()").getall()
        # If no <li> elements, get text from <td> (single director case)
        if not directors:
            directors = response.xpath("//th[contains(text(),'Directed by')]/following-sibling::td//a/text()").getall()
        # Convert list to string
        director_str = ", ".join(directors).strip() if directors else None

        box_office = response.xpath("//th[contains(text(),'Box office')]/following-sibling::td//text()").get()
        
        countries = response.xpath("//th[contains(text(),'Country') or contains(text(),'Countries')]/following-sibling::td//li//text()").getall()
        if not countries:
            countries = response.xpath("//th[contains(text(),'Country') or contains(text(),'Countries')]/following-sibling::td//text()").getall()
        countries_str = ", ".join(countries).strip() if countries else None

        yield {
            'Title': title,
            'Year': year,  
            'Director': director_str,            
            'Countries' : countries_str,
            'Box Office': box_office.strip() if box_office else None,
        }
        

## Starting the Spider ##
Finally, after everything is imported and the spider is configured, we can launch the spider to scrap the website, starting from initial web-page

In [3]:
process = CrawlerProcess()
process.crawl(FilmDetailSpider)
process.start()

2025-02-20 20:22:53 [scrapy.utils.log] INFO: Scrapy 2.12.0 started (bot: scrapybot)
2025-02-20 20:22:53 [scrapy.utils.log] INFO: Versions: lxml 5.3.1.0, libxml2 2.12.9, cssselect 1.2.0, parsel 1.10.0, w3lib 2.3.1, Twisted 24.11.0, Python 3.12.5 (v3.12.5:ff3bc82f7c9, Aug  7 2024, 05:32:06) [Clang 13.0.0 (clang-1300.0.29.30)], pyOpenSSL 25.0.0 (OpenSSL 3.4.1 11 Feb 2025), cryptography 44.0.1, Platform macOS-15.2-arm64-arm-64bit
2025-02-20 20:22:53 [scrapy.addons] INFO: Enabled addons:
[]
2025-02-20 20:22:53 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.selectreactor.SelectReactor
2025-02-20 20:22:53 [scrapy.extensions.telnet] INFO: Telnet Password: 1549f8918b1a61d1
2025-02-20 20:22:53 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.memusage.MemoryUsage',
 'scrapy.extensions.feedexport.FeedExporter',
 'scrapy.extensions.logstats.LogStats']
2025-02-20 20:22:53 [scrapy.crawler]

## Data Preprocessing ##
Now, after we got the json file, the data can be not clean and requires preprocessing. \
For example, we need to properly handle the countries of origin for the movie, and also format the box-office

In [4]:
import pandas as pd


df = pd.read_json('data.json')
print(df.columns)
df.head()

Index(['Title', 'Year', 'Director', 'Countries', 'Box Office'], dtype='object')


,Title,Year,Director,Countries,Box Office
0,Avatar,2009,James Cameron,"United Kingdom, [, 2, ], United States, [, 2, ]",$2.923 billion
1,Pirates of the Caribbean: Dead Man's Chest,2006,Gore Verbinski,United States,$1.066 billion
2,Aladdin,2019,Guy Ritchie,United States,$1.054 billion
3,Rogue One: A Star Wars Story,2016,Gareth Edwards,United States,$1.059 billion
4,Moana 2,2024,"David Derrick Jr., [, 1, ], Jason Hand, [, 1, ...",United States,$1.05 billion


### Preprocessing Countries ###
If you involve .head() on the DataFrame, you will be observe that several entries for countries column is not clear. (e.g. for Avatar we have "United Kingdom, [, 2,] ....") \
To deal with that, let manually extract the presented countries in the list, and input the correct values for each movie

In [5]:
countries = [] # This will include the proper countries
for idx, row in df.iterrows():
    countries_individ = str(row[df.columns[3]]).strip()
    countries.append(countries_individ) # Iterate through the rows and add all possible countries in the table
countries_unique = set(countries) # Get unique values for each country
s= ''.join(str(x) for x in list(countries_unique)) # will be needed in future
cleaned = set([x for x in s.split() if len(x) > 2]) # now we can obsere more cleaned countries list
# here we have to manually print this set, and define proper countries
countries = ['China', 'Germany', 'United Kingdom', 'United States', 'New Zealand']

countries_table = [] # now let's create another column with proper countries
for idx, row in df.iterrows(): # iterate over rows
    processed_countries = []
    countries_individ = str(row[df.columns[3]])
    for countr in countries: # iterate over each proper country in the given row
        for s in countr.split(" "): # We split the string with country to extract keywords
            if s in countries_individ and s != 'United': # if the keywords present in this row, add this country to the new entry for that row
                processed_countries.append(countr) # != 'United' because there are UK and USA, to distinguish.
    countries_table.append(','.join(set(processed_countries)))
    
df['Countries'] = countries_table #Finally, change the columns with proper one

Observe the dataset again, perform some small changes

In [7]:
df.columns[0]
df.head(50)
df.at[4, 'Director'] = 'David Derrick Jr, Jason Hand, Dana Ledoux Miller' # The only 1 entry with bad Director names.
df.loc[4]['Director']


'David Derrick Jr, Jason Hand, Dana Ledoux Miller'

In [8]:
df.head(50)

,Title,Year,Director,Countries,Box Office
0,Avatar,2009,James Cameron,"United Kingdom,United States",$2.923 billion
1,Pirates of the Caribbean: Dead Man's Chest,2006,Gore Verbinski,United States,$1.066 billion
2,Aladdin,2019,Guy Ritchie,United States,$1.054 billion
3,Rogue One: A Star Wars Story,2016,Gareth Edwards,United States,$1.059 billion
4,Moana 2,2024,"David Derrick Jr, Jason Hand, Dana Ledoux Miller",United States,$1.05 billion
5,Toy Story 3,2010,Lee Unkrich,United States,$1.067 billion
6,Star Wars: Episode I – The Phantom Menace,1999,George Lucas,United States,$1.047 billion
7,Pirates of the Caribbean: On Stranger Tides,2011,Rob Marshall,United States,$1.046 billion
8,Toy Story 4,2019,Josh Cooley,United States,$1.074 billion
9,Despicable Me 3,2017,"Pierre Coffin, Kyle Balda",United States,$1.035


### Handle the Box-office ###
Now, let's handle the box-office: make the integers correct, and remove dollar signs 

In [9]:
import re


box_office_cleaned = []
for idx, row in df.iterrows(): # iterate over rows
    budget = str(row[df.columns[-1]]).strip() # fetch the box office
    numb_budget = re.findall(r'\d+', budget) # extract the numeric value
    box_office_cleaned.append(numb_budget[0] + numb_budget[1] + '0' * (9 - len(numb_budget[1]))) # add needed zeros (according to billion)
    
df.columns
df['Box Office'] = box_office_cleaned # change column with proper one
df.head(5) 

,Title,Year,Director,Countries,Box Office
0,Avatar,2009,James Cameron,"United Kingdom,United States",2923000000
1,Pirates of the Caribbean: Dead Man's Chest,2006,Gore Verbinski,United States,1066000000
2,Aladdin,2019,Guy Ritchie,United States,1054000000
3,Rogue One: A Star Wars Story,2016,Gareth Edwards,United States,1059000000
4,Moana 2,2024,"David Derrick Jr, Jason Hand, Dana Ledoux Miller",United States,1050000000


## Create a Database ## 
Now, let's deal with database. For this Assignment I will use SQLite, since it is great choice for educational project, and is very easy.
Let's start with importing the required libraries

In [10]:
from sqlalchemy import create_engine, ForeignKey, Column, String, Integer, CHAR, MetaData, Table
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker


Let's declare the table, which we will populate with our information

In [11]:
Base = declarative_base()

class Film(Base):
    __tablename__ = "films"
    
    
    id = Column("id", Integer, primary_key=True, autoincrement=True)
    title = Column("title", String, nullable=False)
    release_year = Column("release_year", Integer)
    director = Column("director", String)
    box_office = Column("box_office", String)
    country = Column("country", String)
    
    def __init__(self, title, release_year, director, country, box_office):
        self.title = title
        self.release_year = release_year
        self.director = director
        self.box_office = box_office
        self.country = country


2025-02-20 20:23:37 [py.warnings] WARNING: /var/folders/yv/pfr0hn755gjbm9trsh5xk0ch0000gn/T/ipykernel_18597/786173885.py:1: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()



Let's create engine, create tables, and populate it with our scrapped dataset.

In [12]:
engine = create_engine("sqlite:///films.db")
Base.metadata.create_all(bind=engine)


Session = sessionmaker(bind=engine)
session = Session()

for idx, row in df.iterrows():
    film_attributes = [row[df.columns[i]] for i in range(len(df.columns))]
    film = Film(film_attributes[0], film_attributes[1], film_attributes[2], film_attributes[3], film_attributes[4])
    session.add(film)

session.commit()

Finally, let's export the database content to the json file, as stated in the assignment

In [13]:
import json

metadata = MetaData()
metadata.reflect(bind=engine)

table_name = "films"
table = metadata.tables[table_name]

with engine.connect() as connection:
    result = connection.execute(table.select()).mappings().all()

data_list = [dict(row) for row in result]

json_file = "movies.json"
with open(json_file, "w", encoding="utf-8") as f:
    json.dump(data_list, f, indent=4)

